In [3]:
from nltk.tokenize import RegexpTokenizer
import re
from collections import defaultdict # for the model
from nltk.util import ngrams
import pandas as pd

## Load the data

In [10]:
# you need your machine's path 
# use * for the folder that has all the .txt files

train_pos_filenames = glob.glob('/Users/jasonmaloney/Documents/Syracuse/IST 736 Text Mining/Text Mining Project/aclImdb/train/pos/*.txt')
train_neg_filenames = glob.glob('/Users/jasonmaloney/Documents/Syracuse/IST 736 Text Mining/Text Mining Project/aclImdb/train/neg/*.txt')
test_pos_filenames = glob.glob('/Users/jasonmaloney/Documents/Syracuse/IST 736 Text Mining/Text Mining Project/aclImdb/test/pos/*.txt')
test_neg_filenames = glob.glob('/Users/jasonmaloney/Documents/Syracuse/IST 736 Text Mining/Text Mining Project/aclImdb/test/neg/*.txt')
print(len(train_pos_filenames))
print(len(train_neg_filenames))
print(len(test_pos_filenames))
print(len(test_neg_filenames))

12500
12500
12500
12500


In [11]:
# open files
train_pos_text = []
for filename in train_pos_filenames[:1000]:
    with open(filename, encoding='utf-8') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'\n|<\w+\s/>', '', text)
        train_pos_text.append(text)
print("train_pos_text:")
print(train_pos_text[0])


train_neg_text = []
for filename in train_neg_filenames[:1000]:
    with open(filename, encoding='utf-8') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'\n|<\w+\s/>', '', text)
        train_neg_text.append(text)
print("train_neg_text:")
print(train_neg_text[0])

test_pos_text = []
for filename in test_pos_filenames[:1000]:
    with open(filename, encoding='utf-8') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'\n|<\w+\s/>', '', text)
        test_pos_text.append(text)
print("test_pos_text:")
print(test_pos_text[0])

test_neg_text = []
for filename in test_neg_filenames[:1000]:
    with open(filename, encoding='utf-8') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'\n|<\w+\s/>', '', text)
        test_neg_text.append(text)
print("test_neg_text:")
print(test_neg_text[0])

train_pos_text:
for a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. imagine a movie where joe piscopo is actually funny! maureen stapleton is a scene stealer. the moroni character is an absolute scream. watch for alan "the skipper" hale jr. as a police sgt.
train_neg_text:
working with one of the best shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.branagh steals the film from under fishburne's nose, and there's a talented cast on good form.
test_pos_text:
based on an actual story, john boorman shows the struggle of an american doctor, whose husband and son were murdered and she was continually plagued with her loss. a holiday to burma with her sister seemed like a good idea to get away from it all, but when her passport was stolen in rangoon, she could not leave the country with her sister, and was forced to stay back until she could get i.d. papers from the american embassy. 

In [13]:
# get as a big long string
words = ''
for rev in train_pos_text:
    words += ' ' + rev

for rev in train_neg_text:
    words += ' ' + rev
    
for rev in test_pos_text:
    words += ' ' + rev

for rev in test_neg_text:
    words += ' ' + rev

# define tokenizer to get words
tokenizer = RegexpTokenizer(r'\w+')

# words has all the review as one string
tokens = tokenizer.tokenize(words)

## Bigram Model

In [15]:
# create a dictionary of dictionaries
# the inner function will take on counts (frequencies) 
# for probabilities
bigram_model = defaultdict(lambda: defaultdict(lambda: 0))

In [16]:
# fill up the dictionary
for word1, word2 in ngrams(tokens, 2):
     # increase the count (frequency of tokens)
    bigram_model[word1][word2] += 1 

In [17]:
# transform the counts into probabilities
for word1 in bigram_model:
    #print(word1)
    # sum freq of all the first 3 word combos of each phrase
    total_count = float(sum(bigram_model[word1].values()))
    # get each fourth word and divide each frequency by total count
    # for conditional probabilities
    for word2 in bigram_model[word1]:
        # get the probability
        # four_gram_model[word1_word2_word3]/total_count
        bigram_model[word1][word2] /= total_count

In [18]:
# function to return key for any value 
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

## Interactive User Input

In [106]:
user_text = input('Please enter a word: ')
user_tokens = tokenizer.tokenize(user_text)
prob_list = bigram_model.get(user_tokens[0]).values()
most_likely = max(prob_list)
getKeysByValue(dict(bigram_model[user_tokens[0]]), most_likely)

Please enter a word: i


['was']

In [105]:
# get highest probability of the next word prediction
max(bigram_model.get('a').values())

0.02272288877743914

## Trigram Model

In [21]:
# create a dictionary of dictionaries
# the inner function will take on counts (frequencies) 
# for probabilities
trigram_model = defaultdict(lambda: defaultdict(lambda: 0))

In [22]:
# fill up the dictionary
for word1, word2, word3 in ngrams(tokens, 3):
     # increase the count (frequency of tokens)
    trigram_model[word1, word2][word3] += 1 

In [23]:
# transform the counts into probabilities
for word1_word2 in trigram_model:
    #print(word1_word2_word3)
    # sum freq of all the first 3 word combos of each phrase
    total_count = float(sum(trigram_model[word1_word2].values()))
    # get each third word and divide each frequency by total count
    # for conditional probabilities
    for word3 in trigram_model[word1_word2]:
        # get the probability
        # four_gram_model[word1_word2_word3]/total_count
        trigram_model[word1_word2][word3] /= total_count

## Interactive User Input

In [54]:
user_text = input('Please enter two words: ')
user_tokens = tokenizer.tokenize(user_text)
#print (user_tokens)
prob_list = trigram_model[user_tokens[0], user_tokens[1]].values()
most_likely = max(prob_list)
getKeysByValue(dict(trigram_model[user_tokens[0], user_tokens[1]]), most_likely)

Please enter two words: robert deniro


['and']

## 4-gram model 

In [55]:
# create a dictionary of dictionaries
# the inner function will take on counts (frequencies) 
# for probabilities
four_gram_model = defaultdict(lambda: defaultdict(lambda: 0))

In [56]:
# fill up the dictionary
for word1, word2, word3, word4 in ngrams(tokens, 4):
     # increase the count (frequency of tokens)
    four_gram_model[word1, word2, word3][word4] += 1 

In [57]:
# transform the counts into probabilities
for word1_word2_word3 in four_gram_model:
    #print(word1_word2_word3)
    # sum freq of all the first 3 word combos of each phrase
    total_count = float(sum(four_gram_model[word1_word2_word3].values()))
    # get each fourth word and divide each frequency by total count
    # for conditional probabilities
    for word4 in four_gram_model[word1_word2_word3]:
        # get the probability
        # four_gram_model[word1_word2_word3]/total_count
        four_gram_model[word1_word2_word3][word4] /= total_count

## Interactive User Input

In [36]:
max(dict(four_gram_model['the', 'movie', 'is']).values())

0.04838709677419355

In [64]:
user_text = input('Please enter three words: ')
user_tokens = tokenizer.tokenize(user_text)
#print (user_tokens)
prob_list = four_gram_model[user_tokens[0], user_tokens[1], user_tokens[2]].values()
most_likely = max(prob_list)
getKeysByValue(dict(four_gram_model[user_tokens[0], user_tokens[1], user_tokens[2]]), most_likely)

Please enter three words: robert deniro and


['jane', 'eddie']

# 5-gram Model

In [86]:
five_gram_model = defaultdict(lambda: defaultdict(lambda: 0))

In [87]:
for word1, word2, word3, word4, word5 in ngrams(tokens, 5):
    # increase the count (frequency for tokens)
    five_gram_model[word1, word2, word3, word4][word5] += 1 

In [89]:
for word1_word2_word3_word4 in five_gram_model:
    total_count = float(sum(five_gram_model[word1_word2_word3_word4].values()))
    for word5 in five_gram_model[word1_word2_word3_word4]:
        # get the probability
        five_gram_model[word1_word2_word3_word4][word5] /= total_count


## Interactive User Input

In [92]:
user_text = input('Please enter four words: ')
user_tokens = tokenizer.tokenize(user_text)
print (len(user_tokens))
prob_list = five_gram_model[user_tokens[0], user_tokens[1], user_tokens[2], user_tokens[3]].values()
most_likely = max(prob_list)
getKeysByValue(dict(five_gram_model[user_tokens[0], user_tokens[1], user_tokens[2], user_tokens[3]]), most_likely)

Please enter four words: oi oi oi oi
4


ValueError: max() arg is an empty sequence

Would like to have it be ----
'Enter 1 - 4 words please:'
The model would tokenize the phrase 
According to length, apply the appropriate model

In [79]:
train_pos_text[0]

'for a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. imagine a movie where joe piscopo is actually funny! maureen stapleton is a scene stealer. the moroni character is an absolute scream. watch for alan "the skipper" hale jr. as a police sgt.'

In [133]:
###### create a function that prompts for words...input
###### into the proper model based on length
###### if max( ) error - print a message

# prompt user input
user_text = input("Please enter 1 to 4 words of your review: ")
user_tokens = tokenizer.tokenize(user_text)
# check the length condition
if len(user_tokens) > 4:
    print("You have entered too many words, please enter 1 to 4 words of your review: ")
elif len(user_tokens) == 1:
    # bigram model
    # use a try and except statement for nicer output of errors.
    try:
        prob_list = bigram_model.get(user_tokens[0]).values()
        most_likely = max(prob_list)
        next_word = getKeysByValue(dict(bigram_model[user_tokens[0]]), most_likely)
        print('Your predicted phrase is:', user_tokens[0], next_word)
    except AttributeError or ValueError:
        print('Sorry that word is not in our vocabulary')
elif len(user_tokens) == 2:
    # trigram model
    try:
        prob_list = trigram_model[user_tokens[0], user_tokens[1]].values()
        most_likely = max(prob_list)
        next_word = getKeysByValue(dict(trigram_model[user_tokens[0], user_tokens[1]]), most_likely)
        print('Your predicted phrase is:', user_tokens[0], user_tokens[1], next_word)
    except ValueError:
        print('That phrase is not in our vocabulary.')
elif len(user_tokens) == 3:
    # 4 gram model
    try:
        prob_list = four_gram_model[user_tokens[0], user_tokens[1], user_tokens[2]].values()
        most_likely = max(prob_list)
        next_word = getKeysByValue(dict(four_gram_model[user_tokens[0], user_tokens[1], user_tokens[2]]), most_likely)
        print('Your predicted phrase is:', user_tokens[0], user_tokens[1], user_tokens[2], next_word)
    except ValueError:
        print('Sorry, that phrase is not in our vocabulary')
elif len(user_tokens) == 4:
    # 5 gram model
    try:
        prob_list = five_gram_model[user_tokens[0], user_tokens[1], user_tokens[2], user_tokens[3]].values()
        most_likely = max(prob_list)
        next_word = getKeysByValue(dict(five_gram_model[user_tokens[0], user_tokens[1], user_tokens[2], user_tokens[3]]), most_likely)
        print('Your predicted phrase is:', user_tokens[0], user_tokens[1], user_tokens[2], user_tokens[3], next_word)
    except ValueError:
        print('Sorry that phrase is not in our vocabulary')


Please enter 1 to 4 words of your review: oi
Sorry that word is not in our vocabulary


In [118]:
prob_list = bigram_model.get('hi').values()
most_likely = max(prob_list)
getKeysByValue(dict(bigram_model['hi']), most_likely)

['def']